In [1]:
!pip install sqlalchemy>=2.0
!pip install ipython-sql
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 180.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 152.4 MB/s eta 0:00:00


### Load SQL Extension, Connect to Database

In [2]:
%reload_ext sql

In [3]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
%sql sqlite:///my_data1.db

In [5]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [6]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### Display the names of the unique launch sites



In [7]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [8]:
%%sql
SELECT
    *
FROM
    SPACEXTABLE
LIMIT 1;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)


In [9]:
%%sql
SELECT DISTINCT
    Launch_Site
FROM
    SPACEXTABLE
ORDER BY
    Launch_Site ASC;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
CCAFS SLC-40
KSC LC-39A
VAFB SLC-4E



### 5 records where launch sites begin with the string 'KSC' 


In [10]:
%%sql
SELECT
    *
FROM
    SPACEXTABLE
Where
    Launch_Site Like 'KSC%'
Order by
    Date
Limit 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2017-02-19,14:39:00,F9 FT B1031.1,KSC LC-39A,SpaceX CRS-10,2490,LEO (ISS),NASA (CRS),Success,Success (ground pad)
2017-03-16,6:00:00,F9 FT B1030,KSC LC-39A,EchoStar 23,5600,GTO,EchoStar,Success,No attempt
2017-03-30,22:27:00,F9 FT B1021.2,KSC LC-39A,SES-10,5300,GTO,SES,Success,Success (drone ship)
2017-05-01,11:15:00,F9 FT B1032.1,KSC LC-39A,NROL-76,5300,LEO,NRO,Success,Success (ground pad)
2017-05-15,23:21:00,F9 FT B1034,KSC LC-39A,Inmarsat-5 F4,6070,GTO,Inmarsat,Success,No attempt


### Total payload mass carried by boosters launched by NASA (CRS)

In [11]:
%%sql
SELECT
    Customer,
    sum(PAYLOAD_MASS__KG_) as total_payload_mass
FROM
    SPACEXTABLE
WHERE
    Customer = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


Customer,total_payload_mass
NASA (CRS),45596



### Average payload mass carried by booster version F9 v1.1


In [14]:
%%sql
SELECT
    Booster_Version,
    avg(PAYLOAD_MASS__KG_) as avg_payload_mass
FROM
    SPACEXTABLE
WHERE
    Booster_Version = 'F9 v1.1';

 * sqlite:///my_data1.db
Done.


Booster_Version,avg_payload_mass
F9 v1.1,2928.4


### First date to achieve a successful landing outcome in drone ship



In [15]:
%%sql
SELECT
    Date,
    Landing_Outcome
FROM
    SPACEXTABLE
WHERE
    Landing_Outcome like 'Success%'
    AND Landing_Outcome like '%drone ship%'
ORDER BY
    DATE ASC;

 * sqlite:///my_data1.db
Done.


Date,Landing_Outcome
2016-04-08,Success (drone ship)
2016-05-06,Success (drone ship)
2016-05-27,Success (drone ship)
2016-08-14,Success (drone ship)
2017-01-14,Success (drone ship)
2017-03-30,Success (drone ship)
2017-06-23,Success (drone ship)
2017-06-25,Success (drone ship)
2017-08-24,Success (drone ship)
2017-10-09,Success (drone ship)


In [17]:
%%sql
SELECT
    min(Date),
    Landing_Outcome
FROM
    SPACEXTABLE
WHERE
    Landing_Outcome like 'Success%'
    AND Landing_Outcome like '%drone ship%';

 * sqlite:///my_data1.db
Done.


min(Date),Landing_Outcome
2016-04-08,Success (drone ship)


### Boosters which have success in ground pad with a payload mass greater than 4000 but less than 6000


In [23]:
%%sql
SELECT
    Date,
    Landing_Outcome,
    Booster_Version,
    PAYLOAD_MASS__KG_
FROM
    SPACEXTABLE
WHERE
    Landing_Outcome like 'Success%'
    AND Landing_Outcome like '%ground pad%'
    AND PAYLOAD_MASS__KG_ between 4000 and 6000;

 * sqlite:///my_data1.db
Done.


Date,Landing_Outcome,Booster_Version,PAYLOAD_MASS__KG_
2017-05-01,Success (ground pad),F9 FT B1032.1,5300
2017-09-07,Success (ground pad),F9 B4 B1040.1,4990
2018-01-08,Success (ground pad),F9 B4 B1043.1,5000


### Total number of successful and failure mission outcomes


In [26]:
%%sql
Select
    Mission_Outcome,
    count(*) as num_missions
From
    SPACEXTABLE
Group by
    Mission_Outcome
Order by
    num_missions desc;

 * sqlite:///my_data1.db
Done.


Mission_Outcome,num_missions
Success,98
Success (payload status unclear),1
Success,1
Failure (in flight),1


In [27]:
%%sql
SELECT
    CASE
        WHEN Mission_Outcome LIKE 'Success%' THEN 'Success'
        ELSE 'Failure'
    END AS mission_outcome,
    COUNT(*) AS num_missions
FROM
    SPACEXTABLE
GROUP BY
    CASE
        WHEN Mission_Outcome LIKE 'Success%' THEN 'Success'
        ELSE 'Failure'
    END
ORDER BY
    num_missions DESC;

 * sqlite:///my_data1.db
Done.


mission_outcome,num_missions
Success,100
Failure,1


### All booster_versions that have carried the maximum payload mass


In [28]:
%%sql
Select
    Booster_Version,
    PAYLOAD_MASS__KG_
From
    SPACEXTABLE
Where
    PAYLOAD_MASS__KG_ = (Select max(PAYLOAD_MASS__KG_) from SPACEXTABLE)
Order by
    Booster_Version ASC;

 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 B5 B1048.4,15600
F9 B5 B1048.5,15600
F9 B5 B1049.4,15600
F9 B5 B1049.5,15600
F9 B5 B1049.7,15600
F9 B5 B1051.3,15600
F9 B5 B1051.4,15600
F9 B5 B1051.6,15600
F9 B5 B1056.4,15600
F9 B5 B1058.3,15600


### Display month names, succesful landing_outcomes in ground pad ,booster versions, launch_site for the year 2017

In [31]:
%%sql
SELECT
    substr(Date,0,5) as Year,
    Case substr(Date,6,2)
        When '01' then 'January'
        When '02' then 'February'
        When '03' then 'March'
        When '04' then 'April'
        When '05' then 'May'
        When '06' then 'June'
        When '07' then 'July'
        When '08' then 'August'
        When '09' then 'September'
        When '10' then 'October'
        When '11' then 'November'
        When '12' then 'December'
    End as MonthName,
    Landing_Outcome,
    Booster_Version,
    Launch_Site 
From
    SPACEXTABLE
Where
    substr(Date,0,5) = '2017'
    And Landing_Outcome like 'Success%'
    And Landing_Outcome like '%ground pad%'
Order by
    substr(Date,6,2);


 * sqlite:///my_data1.db
Done.


Year,MonthName,Landing_Outcome,Booster_Version,Launch_Site
2017,February,Success (ground pad),F9 FT B1031.1,KSC LC-39A
2017,May,Success (ground pad),F9 FT B1032.1,KSC LC-39A
2017,June,Success (ground pad),F9 FT B1035.1,KSC LC-39A
2017,August,Success (ground pad),F9 B4 B1039.1,KSC LC-39A
2017,September,Success (ground pad),F9 B4 B1040.1,KSC LC-39A
2017,December,Success (ground pad),F9 FT B1035.2,CCAFS SLC-40


### Count of landing outcomes between the date 2010-06-04 and 2017-03-20, in descending order


In [33]:
%%sql
Select
    Landing_Outcome,
    Count(*) as outcome_count
From
    SPACEXTABLE
Where
    Date >= '2010-06-04'
    And Date <= '2017-03-20'
Group by
    Landing_Outcome
Order by
    outcome_count Desc;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,outcome_count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1


<!--## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |
--!>
